 # 📌ACTIVIDAD 3: CLASIFICADOR K-NN.
 ## PASO 1: CARGA DE DATOS Y CREACIÓN DEL MODELO.
 Crea el notebook saa_02_p04_a3_<tus_iniciales>.ipynb donde realizar esta actividad. Continuamos
usando el fichero wine.csv para implementar ahora un clasificador KNN multiclase y tras construirlo
volvemos a intentar mejorarlo eliminando características que no aporten demasiado, pero usando otro
enfoque, porque ahora no podemos aplicar restricciones de tipo L1.

a) Copia el fichero saa_u02_p04_a2_<tus_iniciales>.ipynb en el fichero
saa_u02_p04_a3_<tus_iniciales>.ipynb para cargar los datos de manera similar a como lo
hiciste antes y completa el resto de apartados. Cuando particiones los datos y realices otras
operaciones aleatorias vuelve a utilizar una semilla aleatoria que coincida con la longitud de tu
nombre concatenada a la de tu primer apellido y concatenada a la de tu segundo apellido.

 b) Continuamos usando un objeto multiclass.OneVsRestClassifier() para que utilice el
método uno contra el resto y ahora un modelo de clasificación de tipo
neighbors.NeighborsClassifier con el valor de k (parámetro n_neighbors) que prefieras.
Borra el código que imprime los parámetros de las líneas de los modelos de regresión porque k
vecinos cercanos no usa línea (borra el apartado c) anterior).

 c) Entrena el modelo y muestra los valores de la matriz de confusión y las métricas de eficiencia o
alternativamente un informe de clasificación donde aparezcan tanto para los datos de train
como para los datos de test.

 d)  Muestra la curva ROC y el valor AUC de cada clase y del modelo en global en los datos de
test. Para hacerlo ten en cuenta que ahora este modelo no implementa la función
decision_function() así que tendremos que utilizar predict_proba().

 e) Responde a la vista de los resultados de los apartados d) y e) ¿Generaliza bien o tiene
overfitting?

 f) Borra el código del resto de apartados anteriores.

## PASO 2: MEJORA DEL MODELO CON SBS.
 La selección secuencial de características (SBS) es una familia de algoritmos de búsqueda de tipo
greedy1 (sistemáticos o ansiosos) que se utilizan para reducir la dimensionalidad d de un espacio de
datos a una dimensión k donde k < d. El objetivo es seleccionar las k características que sean más
relevantes para el problema de entre las que hay originalmente. Esta técnica puede ser muy útil sobre
todo para aquellos algoritmos que no soportan regularización.


 Uno de estos algoritmos es el Sequential Backward Selection (SBS), que introduce un poco de
sobrecarga para seleccionar estas características a cambio de mejorar mucho el rendimiento de su
entrenamiento y funcionamiento.


 La idea del algoritmo SBS es bastante simple: elimina características secuencialmente de los datos
actuales hasta alcanzar el número de características deseado. Para decidir la característica a eliminar
en cada etapa debemos usar una función criterio que llamamos J y que hay que minimizar.


El criterio calculado por la función J puede ser simplemente la diferencia de eficiencia del modelo
antes y después de eliminar la característica. Así que en cada paso eliminamos la característica que
menos pérdida de rendimiento genere. El algoritmo será:
 1. Inicializar k = d donde d es la dimensionalidad de todo el espacio de características de X.
 2. Encontrar la característica x− que maximiza el criterio x− = argmax J(Xk-x) donde x ϵ Xk

---

 1Algoritmos de tipo Greedy: hacen búsquedas escogiendo opciones localmente óptimas y en general consiguen
encontrar soluciones no siempre óptimas a los problemas en un tiempo razonable, en contraste con los algoritmos
exhaustivos que encuentran algunas de las soluciones óptimas de los problemas pero con un esfuerzo muchísimo
mayor.


 3. Eliminar la característica x− del conjunto de características: Xk-1 = Xk – x-; k= k-1
 4. Terminar si K es el número de características deseadas o sino volver al paso 2

 g)
 Aunque está implementado en
scikit-learn
 , como es sencillo de hacer lo vamos a progrmar
nosotros. Así que manos a la obra, crea el fichero
código:

In [1]:
from sklearn.base import clone
from itertools import combinations
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

class SBS_samartlop():
    def __init__(self, estimator, k_features, scoring=accuracy_score, test_size=0.25, random_state=1):
        self.scoring = scoring
        self.estimator = clone(estimator)
        self.k_features = k_features
        self.test_size = test_size
        self.random_state = random_state

    def fit(self, X, y):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=self.test_size, random_state=self.random_state)
        dim = X_train.shape[1]
        self.indices_ = tuple(range(dim))
        self.subsets_ = [self.indices_]
        score = self._calc_score(X_train, y_train, X_test, y_test, self.indices_)  # Desempeño con todas las características
        self.scores_ = [score]

        while dim > self.k_features:
            scores = []
            subsets = []

            for p in combinations(self.indices_, r=dim - 1):
                score = self._calc_score(X_train, y_train, X_test, y_test, p)
                scores.append(score)
                subsets.append(p)

            best = np.argmax(scores)
            self.indices_ = subsets[best]
            self.subsets_.append(self.indices_)
            dim -= 1
            self.scores_.append(scores[best])

        self.k_score_ = self.scores_[-1]
        return self


In [2]:
def transform(self, X):
    return X[:, self.indices_]

def _calc_score(self, x_train, y_train, x_test, y_test, indices):
    self.estimator.fit(x_train[:, indices], y_train)
    y_pred = self.estimator.predict(x_test[:, indices])
    score = self.scoring(y_test, y_pred)
    return score


Para utilizarlo, un ejemplo:


In [3]:
import matplotlib.pyplot as plt
from SBS_samartlop import SBS
knn = KneighborsClassifier(n_neighbors=5)
sbs = SBS(knn, k_features=1)
sbs.fit(X_train_std, y_train)

ModuleNotFoundError: No module named 'SBS_samartlop'

Aunque esta implementación de SBS ya divide internamente el dataset en train y test dentro de su
función fit(), externamente le proporcionamos como datos de trabajo el dataset X_train. Así es como
si internamente la clase SBS crease una división train + val + test para prevenir usar los datos de test
durante el entrenamiento o la búsqueda de parámetros, ya que SBS calcula scores de las mejores

 características y si usamos el dataset test original podemos crear overfitting a esos datos de test y
generar una medida de desempeño engañosa cuando usemos sus datos para comprobar si el modelo
generaliza bien.


 h)
 Como el objeto
sbs
 ha ido coleccionando los
scores
 de cada etapa, podemos graficar la
evolución a medida que va quitando características. Adapta el siguiente código y lo añades a
tu fichero (ten en cuenta que estás usando el objeto multiclase) y cuando lo ejecutes, a la vista
del gráfico, ¿En qué rango de características funciona bien el modelo [desde, hasta]?

In [ ]:
import matplotlib.pyplot as plt
from sklearn.neighbors import KneighborsClassifier, KNeighborsClassifier
from sbs_samartlop import SBS
knn = KNeighborsClassifier(n_neighbors=5)
sbs = SBS(knn, k_features=1)
# Le pedimos que pruebe hasta dejas solo 1
sbs.fit(X_train, y_train)
# Dibujamos como cambia el desempeño al cambiar el nº de características
k_carac = [len(k) for k in sbs.subsets_]
plt.plot(k_carac, sbs.scores_, marker='o')
plt.ylim([0.7, 1.02])
plt.ylabel('Accuracy')
plt.xlabel('Nº de características')
plt.grid()
plt.tight_layout()
plt.title("Desempeño vs nº características")
plt.show()

i)
 Si al clasificador
KNN
 le afectan cosas, intenta minimizar estas cosas antes de realizar la
división en
train + test
 . Además, por prudencia (porque nos está dando una medida del
desempeño agrupado en las 3 clases) vamos a quedarnos con la cantidad de características
mayor que tenga un mejor desempeño, es decir, si entre [1, 4] tiene buen desempeño, nos
quedamos con 4 en vez de con 1 por si a alguna clase le afecta demasiado no tener alguna de
las que descartemos, pero esa pérdida se enmascara con buenos resultados en las otra clases.
Así que nos quedaríamos con 4. Pero es necesario saber qué características son esas. Si adaptas
este código en el que veríamos qué puedes saberlo suponiendo que hay 13 y queremos
quedarnos con 3

In [ ]:
k3 = list(sbs.subsets_[10])
print(df_vinos.columns[1:][k3])

j)
 Ahora crea un nuevo clasificador pero descartando las categorías que el método anterior te
indique que son descartables. Puedes adaptar el ejemplo. Calcula matriz, e informe de
clasificación para
train
 y test (mira si generaliza) y la curva
dos modelos. ¿Hay diferencias significativas?

In [ ]:
knn.fit(X_train[:, k3], y_train)


## PASO 3: MEJORA DEL MODELO CON ENSEMBLES.
 . Compara el desempeño de los
Otra aproximación para hacer el mismo trabajo de encontrar características que se puedan descartar es
usar random forest, un método ensemble que ya hemos comentado en esta unidad y que veremos con
más detalle en la siguiente unidad. Al usar random forest suponemos que la importancia media de las
características decrece cuando los cálculos se hacen a partir de todos los árboles del bosque.
 En scikit-learn la implementación de random forest que hay ya recopila información sobre la
importancia de cada característica cuando se construye el bosque y se almacena en la propiedad
feature_importances_ de un RandomForestClassifier.


k)
Adapta el siguiente código que utiliza un
random forest
de 500 árboles para averiguar la
importancia de cada característica y haz un listado de mayor a menor importancia. ¿Coincide
con alguno de los métodos anteriores que hemos usado?

In [ ]:
from sklearn.ensemble import RandomForestClassifier
carac_labels = df_vinos.columns[1:]
forest = RandomForestClassifier(n_estimators=500, random_state=123)
forest.fit(X_train, y_train)
importancias = forest.feature_importances_
indices = np.argsort(importancias)[::-1]

## PASO 4: BUSCAR EL MEJOR MODELO
 Entrena estos clasificadores a ver si consigues mejorar el accuracy y el overfitting. No uses GridSearch,
prefiero que uses manuealmente los modelos. Si quieres, si puedes usar validación cruzada.
• Regresión logística.

 • SoftMax.

 • Perceptrón.

 • Bagging

 • Boosting

 • Voting de 3 modelos

• Stacking de 3 modelos (los que quieras)

 #### ENTREGAR:
 e)
 Código de entrenamiento de modelos y captura de ejecución de test, matriz de confusión,
informe, curva ROC y AUC de cada uno. Guarda el mejor modelo a un fichero.